In [1]:
import pandas as pd

pd.set_option("display.max_columns", 50)

In [2]:
train_data = pd.read_csv(
    "https://storage.googleapis.com/kagglesdsdata/competitions/10211/111096/train.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1700047022&Signature=xD9KTezYtmJBsMHIubIGoF%2BLjZe6u5EEDK6n1FcAzbSr0UZYQjca4TpUsgjYDlZffbX%2F8h%2Bavp2t1fuX%2FM1rqdKU1DrRySTIQhARmVAJxg1MRgHhW4F4fx%2Bx%2BlXFC8vCiEpRTlRBhcTr3m5ZVUiWNL7bUbBTyd6Q5ldl3%2FZkD4DW0XitaPjOxtjk6iMCTdmHHVW8vyvvJEaWuorjaqgwH8H67bRSHY%2F7zijq4rPd1ZNcqf3xukmHjBs3Bm8bTb1DGYd5w%2FqMg4wSCwFtbQodhy3Q%2ByyFwXNzfsZC9wWSJXN7FM2qTFXE7KB8txNDwXP5V7eiM9GOcxNwxU8MY47tHg%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.csv",
    index_col = 0
)
X_train = train_data.drop("SalePrice", axis = 1)
y_train = train_data["SalePrice"]

## EDA

### Missing values

I'll drop columns with more than 10% missing values.

In [3]:
na_counts = X_train.isna().sum()
na_counts = na_counts[na_counts > 0].sort_values(ascending = False)
missing_frame = pd.DataFrame(
    {
        "Missing number" : na_counts,
        "Missing fraction" : na_counts*100/X_train.shape[0]
    }
)

display(missing_frame)

X_train.drop(
    missing_frame.index[missing_frame["Missing fraction"] > 10],
    axis = 1,
    inplace = True
)

,Missing number,Missing fraction
PoolQC,1453,99.520548
MiscFeature,1406,96.301370
Alley,1369,93.767123
Fence,1179,80.753425
MasVnrType,872,59.726027
FireplaceQu,690,47.260274
LotFrontage,259,17.739726
GarageType,81,5.547945
GarageYrBlt,81,5.547945
GarageFinish,81,5.547945


### Numeric columns

In [3]:
numeric_columns = list(X_train.select_dtypes('number').columns)
X_train[numeric_columns].describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


#### Linear correlation

In [6]:
correlation_matrix = X_train[numeric_columns].corr()

temp = correlation_matrix.unstack().to_dict()
corr_pares = {}
for features, corr_coef in temp.items():
    if (
        ((features[1], features[0]) not in corr_pares) 
        and 
        (features[1] != features[0])
    ):
        corr_pares[features] = corr_coef
del temp

corr_pares = pd.Series(
    corr_pares, name="Correlation"
)

Top 10 correlations

In [10]:
corr_pares.abs().sort_values(ascending=False).to_frame().head(10)

,,Correlation
GarageCars,GarageArea,0.882475
YearBuilt,GarageYrBlt,0.825667
GrLivArea,TotRmsAbvGrd,0.825489
TotalBsmtSF,1stFlrSF,0.819530
2ndFlrSF,GrLivArea,0.687501
BedroomAbvGr,TotRmsAbvGrd,0.676620
BsmtFinSF1,BsmtFullBath,0.649212
YearRemodAdd,GarageYrBlt,0.642277
GrLivArea,FullBath,0.630012
2ndFlrSF,TotRmsAbvGrd,0.616423


In [6]:
from sklearn.impute import SimpleImputer